In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier   
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SequentialFeatureSelector as sfs

In [2]:
def SFS_Feature(indep_X,dep_Y):
        SFSlist1=[]
        
        log_model = LogisticRegression(solver='lbfgs')
        RF = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
        
        SFSmodellist=[log_model,RF]
        
        for i in   SFSmodellist:
            log_sfs = sfs(i, n_features_to_select=6, tol=None, direction='backward', scoring=None, cv=None, n_jobs=None)
            log_fit = log_sfs.fit(indep_X, dep_Y)
            log_sfs_feature=log_fit.transform(indep_X)
            SFSlist1.append(log_sfs_feature)
        return SFSlist1
    
def Feature_Name(indep_X,dep_Y):
        SFSlist=[]
        log_model = LogisticRegression(solver='lbfgs')
        RF = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
        
        SFSmodellist1=[log_model,RF]
        
        for i in   SFSmodellist1:
            log_sfs = sfs(i, n_features_to_select=6 , tol=None, direction='forward', scoring=None, cv=None, n_jobs=None)
            log_fit = log_sfs.fit(indep_X, dep_Y)
            log_sfs_feature=log_fit.transform(indep_X)
            op=log_sfs.get_feature_names_out(input_features=None)
            SFSlist.append(op)
        return SFSlist

def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
        
        #Feature Scaling
        #from sklearn.preprocessing import StandardScaler
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)
        
        return X_train, X_test, y_train, y_test
    
def cm_prediction(classifier,X_test):
        y_pred = classifier.predict(X_test)
        
        # Making the Confusion Matrix
        from sklearn.metrics import confusion_matrix
        cm = confusion_matrix(y_test, y_pred)
        
        from sklearn.metrics import accuracy_score 
        from sklearn.metrics import classification_report 
        
        Accuracy=accuracy_score(y_test, y_pred )
        
        report=classification_report(y_test, y_pred)
        return  classifier,Accuracy,report,X_test,y_test,cm

def logistic(X_train,y_train,X_test):
    
        from sklearn.linear_model import LogisticRegression
        classifier = LogisticRegression(random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm      
    
def svm_linear(X_train,y_train,X_test):
                
        from sklearn.svm import SVC
        classifier = SVC(kernel = 'linear', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm
    
def svm_NL(X_train,y_train,X_test):
                
        from sklearn.svm import SVC
        classifier = SVC(kernel = 'rbf', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm

def Navie(X_train,y_train,X_test):   
    
        from sklearn.naive_bayes import GaussianNB
        classifier = GaussianNB()
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm         
    
    
def knn(X_train,y_train,X_test):
           
        from sklearn.neighbors import KNeighborsClassifier
        classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm
    
def Decision(X_train,y_train,X_test):
        
        from sklearn.tree import DecisionTreeClassifier
        classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm      


def random(X_train,y_train,X_test):
        
        from sklearn.ensemble import RandomForestClassifier
        classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm
    

def rfe_classification(acclog, accsvml, accsvmnl, accknn, accnav, accdes, accrf):
    rfedataframe = pd.DataFrame(index=['Logistic','Random'],
                                columns=['Logistic','SVMl','SVMnl','KNN','Navie','Decision','Random'])

    for number, idex in enumerate(rfedataframe.index):
        if number < len(acclog):
            rfedataframe['Logistic'][idex] = acclog[number]
        if number < len(accsvml):
            rfedataframe['SVMl'][idex] = accsvml[number]
        if number < len(accsvmnl):
            rfedataframe['SVMnl'][idex] = accsvmnl[number]
        if number < len(accknn):
            rfedataframe['KNN'][idex] = accknn[number]
        if number < len(accnav):
            rfedataframe['Navie'][idex] = accnav[number]
        if number < len(accdes):
            rfedataframe['Decision'][idex] = accdes[number]
        if number < len(accrf):
            rfedataframe['Random'][idex] = accrf[number]
            
    return rfedataframe


In [3]:
dataset1=pd.read_csv("prep.csv",index_col=None)
df2=dataset1
df2 = pd.get_dummies(df2, drop_first=True)

indep_X=df2.drop('classification_yes', axis=1)
dep_Y=df2['classification_yes']

In [4]:
SFSlist=SFS_Feature(indep_X,dep_Y)       

acclog=[]
accsvml=[]
accsvmnl=[]
accknn=[]
accnav=[]
accdes=[]
accrf=[]

C:\Users\swath\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\swath\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

In [5]:
SFSlist

[array([[  3.        , 148.11267606,   3.07735602,  12.51815562,
           0.        ,   1.        ],
        [  2.        , 148.11267606,   0.7       ,  10.7       ,
           0.        ,   1.        ],
        [  1.        ,  99.        ,   0.6       ,  12.        ,
           0.        ,   1.        ],
        ...,
        [  3.        , 110.        ,   6.        ,   9.1       ,
           1.        ,   0.        ],
        [  0.        , 207.        ,   6.8       ,   8.5       ,
           1.        ,   1.        ],
        [  0.        , 100.        ,   1.        ,  16.3       ,
           0.        ,   1.        ]]),
 array([[ 3.07735602,  4.62724359, 12.51815562,  1.        ,  0.        ,
          0.        ],
        [ 0.7       ,  4.62724359, 10.7       ,  1.        ,  0.        ,
          0.        ],
        [ 0.6       ,  4.4       , 12.        ,  0.        ,  0.        ,
          0.        ],
        ...,
        [ 6.        ,  2.7       ,  9.1       ,  1.        ,  0

In [6]:
Feature_Name=Feature_Name(indep_X,dep_Y)

C:\Users\swath\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\swath\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

In [7]:
Feature_Name

[array(['al', 'su', 'hrmo', 'pcv', 'sg_d', 'pc_normal'], dtype=object),
 array(['al', 'bu', 'pcv', 'sg_c', 'sg_d', 'dm_yes'], dtype=object)]

In [8]:
for i in SFSlist:   
    X_train, X_test, y_train, y_test=split_scalar(i,dep_Y)   
    
        
    classifier,Accuracy,report,X_test,y_test,cm=logistic(X_train,y_train,X_test)
    acclog.append(Accuracy)
    
    classifier,Accuracy,report,X_test,y_test,cm=svm_linear(X_train,y_train,X_test)  
    accsvml.append(Accuracy)
    
    classifier,Accuracy,report,X_test,y_test,cm=svm_NL(X_train,y_train,X_test)  
    accsvmnl.append(Accuracy)
    
    classifier,Accuracy,report,X_test,y_test,cm=knn(X_train,y_train,X_test)  
    accknn.append(Accuracy)
    
    classifier,Accuracy,report,X_test,y_test,cm=Navie(X_train,y_train,X_test)  
    accnav.append(Accuracy)
    
    classifier,Accuracy,report,X_test,y_test,cm=Decision(X_train,y_train,X_test)  
    accdes.append(Accuracy)
    
    classifier,Accuracy,report,X_test,y_test,cm=random(X_train,y_train,X_test)  
    accrf.append(Accuracy)
    
result=rfe_classification(acclog,accsvml,accsvmnl,accknn,accnav,accdes,accrf)

C:\Users\swath\AppData\Local\Temp\ipykernel_10452\2369635306.py:122: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  rfedataframe['Logistic'][idex] = acclog[number]
C:\Users\swath\AppData\Local\Temp\ipykernel_10452\2369635306.py:124: FutureWar

In [9]:
result
#6

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
Logistic,0.99,0.99,0.98,0.98,0.94,0.94,0.99
Random,0.99,0.99,0.97,0.96,0.86,0.98,0.99
